# Load Data

In [ ]:
from proj1_helpers import *
from implementations import *
from cross_validation import *
import numpy as np

In [ ]:
y, tx, ids = load_csv_data('Data/train.csv', False)
test_y, test_tx, test_ids= load_csv_data('Data/test.csv', False)

# Preprocessing

In [ ]:
print(test_y.shape)

# Cross-Validation

In [ ]:
from cross_validation import *
cross_validation_run(tx, y)

# Weight Computation

In [7]:
w, loss = least_squares(y,tx)
print(w, loss)

0.339686809477 [  8.03494315e-05  -7.20202273e-03  -6.05417273e-03  -5.47559066e-04
  -1.93874702e-02   4.73451621e-04  -2.60379054e-02   3.25106300e-01
  -3.80780260e-05  -2.72729930e+00  -2.21220140e-01   9.50794092e-02
   6.40351616e-02   2.73555898e+00  -3.31801229e-04  -9.54325120e-04
   2.74031573e+00  -5.34164922e-04   9.73498609e-04   3.69225052e-03
   3.54487428e-04  -5.43344599e-04  -3.30448035e-01  -1.40800497e-03
   8.31432882e-04   1.02117272e-03  -1.68047416e-03  -5.83664815e-03
  -1.11087997e-02   2.72775924e+00]


# Output Data

In [ ]:
name = 'first-submission'
y_pred = predict_labels(w,test_tx)
print(y_pred)


create_csv_submission(test_ids, y_pred, name)